In [1]:
import requests
import time
import pandas as pd
from datetime import datetime
from config import *

In [39]:
# Pull all in all email campaign ids.

url = "https://a.klaviyo.com/api/campaigns/?filter=equals(messages.channel,'email')"
headers = {
    "accept": "application/json",
    "revision": "2023-10-15",
    "Authorization": "Klaviyo-API-Key " + kp_private_key
}
response = requests.get(url, headers=headers)

# Request test
if response.status_code == 200:
    response = response.json()
else:
    print('Failed to retrieve campaigns: ', response.status_code) 

# Extracting metric data
campaigns = [campaign for campaign in response['data'] if campaign['type'] == 'campaign']

# Output is heavily nested, and contains a lot of useless meta data.
# Just going to cherry pick the values we need into a flattened table.
df_campaigns = pd.DataFrame([{
    'campaign_id': campaign['id'],
    'campaign_name': campaign['attributes']['name'],
    'status': campaign['attributes']['status'],
    'audiences': campaign['attributes']['audiences']['included'],
    'scheduled': campaign['attributes']['scheduled_at'], # Used for metric capture window.
    'sent': campaign['attributes']['send_time'], # Effectively useless depending on smart send times.
} for campaign in campaigns])

# Adding an end date to mark when Klaviyo attribution window ends.
# Will be referenced later when calling for aggregate metrics.
df_campaigns['end_date'] = pd.to_datetime(df_campaigns['sent'])
df_campaigns['end_date'] = df_campaigns['end_date'].apply(lambda x: (x + pd.Timedelta(days=14)).isoformat() if pd.notna(x) else None)

# Limit to only sent emails. No need to pull in Drafts and Upcoming emails.
df_campaigns = df_campaigns[df_campaigns['status'] == 'Sent']
df_campaigns.reset_index(drop=True, inplace=True)

In [9]:
# Pull in unique metric IDs.

url = "https://a.klaviyo.com/api/metrics/"
headers = {
    "accept": "application/json",
    "revision": "2023-10-15",
    "Authorization": "Klaviyo-API-Key " + kp_private_key
}

response = requests.get(url, headers=headers)
    
# Request test
if response.status_code == 200:
    response = response.json()
else:
    print('Failed to retrieve metrics: ', response.status_code) 
    
# Output is heavily nested, and contains a lot of useless meta data.
# Just going to cherry pick the values we need into a flattened table.
metrics = [item for item in response['data'] if item['type'] == 'metric']

# Converting to DataFrame
df_metrics = pd.DataFrame([{
    'metric_id': metric['id'],
    'metric_name': metric['attributes']['name']
} for metric in metrics])

# ID's below are: received email, opened email, clicked email, unsubscribed.
# Non-native Klaviyo values (e.g. placed order) don't seem to populate.
df_metrics = df_metrics[df_metrics['metric_id'].isin(['KenCtn', 'QBEgYP', 'JBmzQK', 'NJB98W'])]
df_metrics.reset_index(drop=True, inplace=True)

In [37]:
# Klaviyo requires that you call every metric for every campaign.
# Going to have to call every row in df_campaign, and cycle through every df_metric id.
# We could just hard code these in a dict, but I imagine the metric requests will grow with time.
# It will return a daily count of values, but we're going to drop the dates and group aggregate values.
# Should be easier to unpivot in a main table later.

def call_api_for_metrics(campaign_row, metric_row, kp_private_key):
    
    campaign_id = campaign_row['campaign_id']
    campaign_name = campaign_row['campaign_name']
    scheduled = campaign_row['scheduled']
    end_date = campaign_row['end_date']

    metric_id = metric_row['metric_id']
    metric_name = metric_row['metric_name']

    # API URL
    url = "https://a.klaviyo.com/api/metric-aggregates/"

    # Preparing the payload
    payload = { "data": {
            "type": "metric-aggregate",
            "attributes": {
                "metric_id": metric_id,
                "measurements": ["unique"], # Can use: sum_value, count, unique
                "interval": "month", # Can use: hour, day, week, month
                "by": ["$message"],
                "filter": [
                    "greater-or-equal(datetime," + scheduled + ")",  
                    "less-than(datetime," + end_date + ")",  
                    "equals($message,\"" + campaign_id+ "\")"],
                "timezone": "US/Pacific"
            }
        } }

    # Headers for the API request
    headers = {
        "accept": "application/json",
        "revision": "2023-10-15",
        "content-type": "application/json",
        "Authorization": "Klaviyo-API-Key " + kp_private_key
    }

    # Making the API request
    response = requests.post(url, json=payload, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        response_data = response.json()
    else:
        print('Failed to retrieve data: ', response.status_code)
        return pd.DataFrame()

    # Extracting metric data
    metric_aggregate = response_data['data']

    # Preparing data for DataFrame
    data_for_df = []
    dates = metric_aggregate['attributes']['dates']
    measurements = metric_aggregate['attributes']['data'][0]['measurements']['unique']

    for date, measurement in zip(dates, measurements):
        data_for_df.append({
            'campaign_id': campaign_id,
            'campaign_name': campaign_name,
            'metric_id': metric_id,
            'metric_name': metric_name,
            'date': date,
            'aggregate': measurement
        })

    return pd.DataFrame(data_for_df)

def main(df_campaigns, df_metrics, kp_private_key):
    final_output = pd.DataFrame()

    for campaign_index, campaign_row in df_campaigns.iterrows():
        for metric_index, metric_row in df_metrics.iterrows():
            result = call_api_for_metrics(campaign_row, metric_row, kp_private_key)
            final_output = pd.concat([final_output, result])

            # Rate limiting: Burst 10/s, Steady 150/m; appears to be a lie.
            time.sleep(1)

    return final_output

# Call main()
metric_output = main(df_campaigns, df_metrics, kp_private_key)

# Remove date columns, sum aggregate values to get a readable table.
metric_output = metric_output.groupby(['campaign_id', 'campaign_name', 'metric_id', 'metric_name'])['aggregate'].sum().reset_index()

In [38]:
# Pivot the output, merge into df_campaigns.
pivot = metric_output.pivot(index='campaign_id', columns='metric_name', values='aggregate')
df_campaigns = df_campaigns.merge(pivot, on='campaign_id', how='left')
df_campaigns

,campaign_id,campaign_name,status,audiences,scheduled,sent,end_date,Clicked Email,Opened Email,Received Email,Unsubscribed
0,01HG8R8PZDC1E0H1VMYP06RRWH,KBE231211 - YouTube,Sent,[TNsGmi],2023-11-30T19:29:16.933491+00:00,2023-12-11T22:00:00+00:00,2023-12-25T22:00:00+00:00,2177.0,18216.0,25001.0,19.0
1,01HGTQ8D37CEBW6MZ586CZ4N56,KBE231209 - Standard Shipping,Sent,[WN6kni],2023-12-07T21:29:26.048383+00:00,2023-12-09T22:00:00+00:00,2023-12-23T22:00:00+00:00,11649.0,215391.0,381719.0,349.0
2,01HFQ7MKBKHEYT63BXSF79958X,KBE231208 - Bundles,Sent,[WN6kni],2023-12-06T23:08:35.307815+00:00,2023-12-08T22:00:00+00:00,2023-12-22T22:00:00+00:00,6046.0,211795.0,382407.0,308.0
3,01HFPZ1FAQ4YRQK8BQ6KBEAF9K,KBE231206 - Vinterblomma,Sent,[WN6kni],2023-11-27T23:28:06.724446+00:00,2023-12-06T22:00:00+00:00,2023-12-20T22:00:00+00:00,7506.0,222941.0,382459.0,343.0
4,01HFCDJT3RR3JNJQGYRAC7P8FK,KBE231204 - Joyce Cardigan,Sent,[TNsGmi],2023-11-22T19:08:55.187005+00:00,2023-12-04T22:00:00+00:00,2023-12-18T22:00:00+00:00,4126.0,19128.0,25221.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...
83,01H647ZTQBW9420K6G60V3N5G0,KBE230807 - Summer Tool Sale,Sent,[VJcEfT],2023-07-26T23:17:59.642488+00:00,2023-08-07T22:00:00+00:00,2023-08-21T22:00:00+00:00,11244.0,208233.0,390799.0,768.0
84,01H5SZKGH5MMEEJJDABVAH1KR2,KBE230805 - Stroll,Sent,[VJcEfT],2023-07-25T22:59:07.769707+00:00,2023-08-05T22:00:00+00:00,2023-08-19T22:00:00+00:00,7809.0,206650.0,391759.0,859.0
85,01H5NASD46Y7GH8CRZEYDXR794,KBE230803 - Tool Sale,Sent,[VJcEfT],2023-08-01T15:50:30.257787+00:00,2023-08-03T08:00:00+00:00,2023-08-17T08:00:00+00:00,9010.0,200306.0,394903.0,1070.0
86,01H5MVSY2BXEREG89F4AFM5WCF,KBE230729 - Lace Fingering,Sent,[VJcEfT],2023-07-21T23:33:09.683968+00:00,2023-07-29T22:00:00+00:00,2023-08-12T22:00:00+00:00,6035.0,229155.0,392045.0,1003.0


In [ ]:
# Pull all tags
url = "https://a.klaviyo.com/api/tags/"
headers = {
    "accept": "application/json",
    "revision": "2023-10-15",
    "Authorization": "Klaviyo-API-Key " + kp_private_key
}

response = requests.get(url, headers=headers)

# Request test
if response.status_code == 200:
    response = response.json()
else:
    print('Failed to retrieve campaigns: ', response.status_code) 
    
# Extracting metric data
tags = [tag for tag in response['data'] if tag['type'] == 'tag']

# Converting to DataFrame
df_tags = pd.DataFrame([{
    'tag_id': tag['id'],
    'name': tag['attributes']['name'],
    'tag_group': tag['relationships']['tag-group']['data']['id']
} for tag in tags])

df_tags.head(25)

In [ ]:
# Pull all tag groups

url = "https://a.klaviyo.com/api/tag-groups/"
headers = {
    "accept": "application/json",
    "revision": "2023-10-15",
    "Authorization": "Klaviyo-API-Key " + kp_private_key
}

response = requests.get(url, headers=headers)

# Request test
if response.status_code == 200:
    response = response.json()
else:
    print('Failed to retrieve tag groups: ', response.status_code) 
    
# Extracting metric data
tag_groups = [tag_group for tag_group in response['data'] if tag_group['type'] == 'tag-group']

# Converting to DataFrame
df_tag_groups = pd.DataFrame([{
    'tag_group_id': tag_group['id'],
    'name': tag_group['attributes']['name']
} for tag_group in tag_groups])

df_tag_groups.head(25)

In [ ]:
# Self Tag Test.
url = "https://a.klaviyo.com/api/campaigns/01HFPZ1FAQ4YRQK8BQ6KBEAF9K/relationships/tags/"
headers = {
    "accept": "application/json",
    "revision": "2023-10-15",
    "Authorization": "Klaviyo-API-Key " + kp_private_key
}

response = requests.get(url, headers=headers)

# Request test
if response.status_code == 200:
    response = response.json()
else:
    print('Failed to retrieve tags: ', response.status_code) 

# Extracting metric data
campaign_tags = [tag for tag in response['data'] if tag['type'] == 'tag']

# Converting to DataFrame
df_campaign_tags = pd.DataFrame([{
    'type': tag['type'],
    'id': tag['id']
} for tag in campaign_tags])

df_campaign_tags.head(3)